In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
!pip install selenium

In [4]:
#10 states links
state_links=["https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/upsrtc",
             "https://www.redbus.in/online-booking/pepsu",
             "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/astc"
]

In [4]:
#open the browser
driver = webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in")
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

#implicit wait
def apsrtc_link_route(route_xpath):
    LINKS_APSRTC = []  # List to store APSRTC bus links
    ROUTE_APSRTC = []  # List to store APSRTC route names

    # Assuming 'paths' is passed as the route_xpath, which is XPath for bus routes.
    # Wait for the route elements to be present (update this XPath based on APSRTC page)
    wait = WebDriverWait(driver, 20)

    try:
        # Loop over multiple pages
        i = 0
        while True:
            # Wait for the route elements on the current page
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))
            
            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_APSRTC.append(route.text)
                LINKS_APSRTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath accordingly)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_APSRTC, ROUTE_APSRTC

# Example route XPath for APSRTC buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"

# Calling the function to get links and routes
LINKS_APSRTC, ROUTE_APSRTC = apsrtc_link_route(route_xpath)


No more pages to paginate at step 0


In [6]:
df_a = pd.DataFrame({"Route_name": ROUTE_APSRTC, "Route_link": LINKS_APSRTC})
df_a

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
5,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
6,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
7,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
8,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
9,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...


In [7]:
df=df_a

In [8]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [9]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [10]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Kamakshi Tours And Travels,Non A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,INR 600,27 Seats available,4.2\n207,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
1,Sri Sahasra Tours and Travels,A/C Seater / Sleeper (2+1),21:00,05:30,08h 30m,INR 560,27 Seats available,4.0\n72,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
2,Sri Vengamamba Bus Transport (SVBT),A/C Seater / Sleeper (2+1),21:40,06:15,08h 35m,INR 654,19 Seats available,4.4\n291,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
3,V Kaveri Travels,Scania AC Multi Axle Sleeper (2+1),21:30,05:45,08h 15m,INR 1200,21 Seats available,4.2\n256,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
4,Cherry tours&travels,Non A/C Seater / Sleeper (2+1),21:50,06:00,08h 10m,475,25 Seats available,4.0\n15,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
...,...,...,...,...,...,...,...,...,...,...
121,Navayuga Travels,A/C Sleeper (2+1),22:00,02:10,04h 10m,INR 690,25 Seats available,3.5,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada
122,Sai RK Travels,Non A/C Seater / Sleeper (2+1),22:40,02:30,03h 50m,INR 545,23 Seats available,4.4,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada
123,Morning Star Travels,A/C Sleeper (2+1),21:10,02:25,05h 15m,INR 600,22 Seats available,2.7,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada
124,Sai Sree Travels,A/C Sleeper (2+1),22:00,02:30,04h 30m,INR 590,25 Seats available,2.8,https://www.redbus.in/bus-tickets/visakhapatna...,Visakhapatnam to Kakinada


In [11]:
df1.to_csv("C:/Users/Shalini/Downloads/AP_data.csv")#done1

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

driver = webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in")
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

def tsrtc_link_route(route_xpath):
    LINKS_TSRTC = []  # List to store TSRTC bus links
    ROUTE_TSRTC = []  # List to store TSRTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_TSRTC.append(route.text)
                LINKS_TSRTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_TSRTC, ROUTE_TSRTC

# Example route XPath for TSRTC buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"

# Calling the function to get links and routes
LINKS_TSRTC, ROUTE_TSRTC = tsrtc_link_route(route_xpath)


No more pages to paginate at step 0


In [13]:
# Create a DataFrame for TSRTC
df_tsrtc = pd.DataFrame({"Route_name": ROUTE_TSRTC,"Route_link": LINKS_TSRTC})
# Display the DataFrame
df_tsrtc


,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [14]:
df=df_tsrtc

In [15]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [16]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [17]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,TSRTC - 7648,Super Luxury (Non AC Seater 2+2 Push Back),09:30,14:00,04h 30m,INR 320,26 Seats available,4.2\n4,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
1,TSRTC - 64362,Super Luxury (Non AC Seater 2+2 Push Back),09:30,14:00,04h 30m,INR 320,28 Seats available,3.6,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
2,TSRTC - 8978,Super Luxury (Non AC Seater 2+2 Push Back),09:40,13:35,03h 55m,INR 320,30 Seats available,3.9\n11,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
3,TSRTC - 8647,RAJDHANI (A.C. Semi Sleeper),10:00,14:00,04h 00m,INR 389,33 Seats available,3.6\n37,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
4,TSRTC - 64824,Super Luxury (Non AC Seater 2+2 Push Back),10:30,14:50,04h 20m,INR 320,25 Seats available,2.0\n10,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad
...,...,...,...,...,...,...,...,...,...,...
131,Kaveri Travels,Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1),21:30,05:45,08h 15m,INR 1000,26 Seats available,4.3\n146,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
132,V Kaveri Travels,A/C Sleeper (2+1),22:00,06:15,08h 15m,INR 1000,26 Seats available,4.1\n73,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
133,LUCKY TRAVELS,A/C Sleeper (2+1),22:45,06:05,07h 20m,INR 1199,27 Seats available,4.0\n18,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole
134,Cherry tours&travels,Non A/C Seater / Sleeper (2+1),21:50,06:00,08h 10m,808,28 Seats available,4.1\n93,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Ongole


In [18]:
df1.to_csv("C:/Users/Shalini/Downloads/tsrtc_data2.csv")#done2

In [19]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

driver = webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in")
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()


def ksrtc_link_route(route_xpath):
    LINKS_KSRTC = []  # List to store KSRTC bus links
    ROUTE_KSRTC = []  # List to store KSRTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_KSRTC.append(route.text)
                LINKS_KSRTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_KSRTC, ROUTE_KSRTC

# Example route XPath for KSRTC buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"

# Calling the function to get links and routes
LINKS_KSRTC, ROUTE_KSRTC = ksrtc_link_route(route_xpath)


No more pages to paginate at step 0


In [20]:
import pandas as pd

# Create a DataFrame for KSRTC
df_ksrtc = pd.DataFrame({"Route_name": ROUTE_KSRTC,"Route_link": LINKS_KSRTC})

# Display the DataFrame (in Jupyter notebook or similar environments)
df_ksrtc


,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


In [21]:
df = df_ksrtc

In [22]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
    time.sleep(2)

    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source

        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()

        time.sleep(5)

        new_page_source = driver_k.page_source

        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Successfully Completed")

Successfully Completed


In [23]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [24]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,SKS Tours & Travels,A/C Sleeper (2+1),21:15,05:45,08h 30m,INR 1199,16 Seats available,4.1\n59,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
1,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,INR 800,14 Seats available,4.1\n285,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
2,NS Transports,VE A/C Sleeper (2+1),22:45,07:30,08h 45m,INR 1200,16 Seats available,4.1\n293,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
3,AdFAST LINE BUS,A/C Sleeper (2+1),21:15,06:15,09h 00m,1069,20 Seats available,4.5\n12,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
4,National travels and logistics,A/C Sleeper (2+1),22:45,07:00,08h 15m,999,13 Seats available,4.3\n19,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...,...
131,DLT BUS,A/C Sleeper (2+1),22:30,05:30,07h 00m,INR 1099,18 Seats available,4.1\n149,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kalpetta (kerala)
132,DLT BUS,A/C Sleeper (2+1),22:30,05:20,06h 50m,INR 1099,18 Seats available,4.5\n58,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kalpetta (kerala)
133,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,07:30,08h 30m,INR 999,10 Seats available,3.9\n92,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kalpetta (kerala)
134,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,07:20,08h 20m,INR 999,10 Seats available,4.2\n64,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kalpetta (kerala)


In [25]:
df1.to_csv("C:/Users/Shalini/Downloads/ksrtc_data3.csv") #done3

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd

# Initialize the WebDriver (make sure you have the appropriate WebDriver installed)
driver = webdriver.Chrome()

# Load the RSRTC webpage
driver.get("https://www.redbus.in")
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

def rsrtc_link_route(route_xpath):
    LINKS_RSRTC = []  # List to store RSRTC bus links
    ROUTE_RSRTC = []  # List to store RSRTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_RSRTC.append(route.text)
                LINKS_RSRTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            
            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_RSRTC, ROUTE_RSRTC

# Example route XPath for RSRTC buses (you may need to inspect the page for the exact route element)
route_xpath_rsrtc = "//a[@class='route']"

# Calling the function to get links and routes
LINKS_RSRTC, ROUTE_RSRTC = rsrtc_link_route(route_xpath_rsrtc)


No more pages to paginate at step 0


In [27]:
import pandas as pd

# Create a DataFrame for RSRTC
df_rsrtc = pd.DataFrame({"Route_name": ROUTE_RSRTC,"Route_link": LINKS_RSRTC})
# Display the DataFrame
(df_rsrtc)

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [28]:
df=df_rsrtc

In [29]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [30]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [31]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),17:10,21:55,04h 45m,INR 600,45 Seats available,4.4\n419,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
1,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),17:00,21:30,04h 30m,INR 550,13 Seats available,4.5\n288,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
2,Jain travels regd,A/C Seater / Sleeper (2+1),21:30,04:30,07h 00m,INR 460,36 Seats available,4.1\n253,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
3,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),12:50,17:20,04h 30m,INR 550,19 Seats available,4.0\n233,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
4,Jakhar Travels,A/C Seater / Sleeper (2+1),22:00,04:30,06h 30m,INR 457,29 Seats available,3.9\n261,https://www.redbus.in/bus-tickets/udaipur-to-j...,Udaipur to Jodhpur
...,...,...,...,...,...,...,...,...,...,...
171,UPSRTC - KSN0275,Ordinary Non AC Seater 2+3,18:06,20:30,02h 24m,INR 121,52 Seats available,3.3,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)
172,V k jain travels,NON A/C Seater/ Sleeper (2+1),14:15,17:01,02h 46m,INR 350,36 Seats available,3.0,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)
173,Jain travels regd,NON AC Seater / Sleeper 2+1,15:45,18:30,02h 45m,INR 250,36 Seats available,3.8,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)
174,Rinku Travels,A/C Seater / Sleeper (2+1),17:30,21:00,03h 30m,INR 350,41 Seats available,3.4,https://www.redbus.in/bus-tickets/kishangarh-t...,Kishangarh to Jaipur (Rajasthan)


In [32]:
df1.to_csv("C:/Users/Shalini/Downloads/rsrtc_data4.csv") #done4

In [33]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
from selenium import webdriver

# Initialize WebDriver
driver = webdriver.Chrome()

# Load the HRTC webpage
driver.get("https://www.redbus.in")
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")

time.sleep(3)
driver.maximize_window()

def hrtc_link_route(route_xpath):
    LINKS_HRTC = []  # List to store HRTC bus links
    ROUTE_HRTC = []  # List to store HRTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_HRTC.append(route.text)
                LINKS_HRTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_HRTC, ROUTE_HRTC

# Example route XPath for HRTC buses (you may need to inspect the page for the exact route element)
route_xpath_hrtc = "//a[@class='route']"

# Calling the function to get links and routes
LINKS_HRTC, ROUTE_HRTC = hrtc_link_route(route_xpath_hrtc)



No more pages to paginate at step 0


In [34]:
# Create a DataFrame for HRTC
df_hrtc = pd.DataFrame({"Route_name": ROUTE_HRTC,"Route_link": LINKS_HRTC})
(df_hrtc)

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
2,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
3,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
4,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
5,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...
6,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
7,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
8,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Delhi to Chamba (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-cha...


In [35]:
df=df_hrtc

In [36]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [37]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [38]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,HRTC - 657,Ordinary 3+2 Non AC Seater,11:00,22:00,11h 00m,INR 512,33 Seats available,2.7\n8,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
1,HRTC - 515,Ordinary 3+2 Non AC Seater,12:30,22:50,10h 20m,INR 512,37 Seats available,4.2,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
2,HRTC - 551,A/C Executive (2+3),15:30,01:30,10h 00m,INR 632,42 Seats available,4.0\n9,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
3,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,03:00,10h 10m,INR 512,26 Seats available,4.2,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
4,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,04:15,10h 16m,INR 512,27 Seats available,2.5\n5,https://www.redbus.in/bus-tickets/delhi-to-shimla,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...,...
94,Laxmi holidays,VE A/C Semi Sleeper (2+2),23:50,05:30,05h 40m,854,37 Seats available,4.6\n88,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Dharamshala (Himachal Pradesh)
95,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:55,06:00,06h 05m,759,28 Seats available,4.7\n293,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Dharamshala (Himachal Pradesh)
96,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),23:55,05:59,06h 04m,759,28 Seats available,4.5\n171,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Dharamshala (Himachal Pradesh)
97,A N Holidays,Bharat Benz A/C Semi Sleeper (2+2),23:59,07:15,07h 16m,599,23 Seats available,4.3\n14,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Dharamshala (Himachal Pradesh)


In [39]:
df1.to_csv("C:/Users/Shalini/Downloads/hrtc_data5.csv") #done5

In [40]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
from selenium import webdriver

# Initialize WebDriver
driver = webdriver.Chrome()

# Load the CTU webpage on Redbus
driver.get("https://www.redbus.in")
driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu/?utm_source=rtchometile")  # Update the URL if necessary

time.sleep(3)
driver.maximize_window()

def ctu_link_route(route_xpath):
    LINKS_CTU = []  # List to store CTU bus links
    ROUTE_CTU = []  # List to store CTU route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_CTU.append(route.text)
                LINKS_CTU.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))  # Adjust as per actual pagination structure

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_CTU, ROUTE_CTU

# Example route XPath for CTU buses (you may need to inspect the page for the exact route element)
route_xpath_ctu = "//a[@class='route']"  # Adjust based on actual page structure

# Calling the function to get links and routes
LINKS_CTU, ROUTE_CTU = ctu_link_route(route_xpath_ctu)


No more pages to paginate at step 0


In [41]:
# Create DataFrame
df_ctu = pd.DataFrame({"Route_name": ROUTE_CTU, "Route_link": LINKS_CTU})
df_ctu

,Route_name,Route_link
0,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
2,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
3,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Ludhiana,https://www.redbus.in/bus-tickets/chandigarh-t...
6,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
7,Chandigarh to Vrindavan,https://www.redbus.in/bus-tickets/chandigarh-t...
8,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Chandigarh to Pathankot,https://www.redbus.in/bus-tickets/chandigarh-t...


In [42]:
df =df_ctu

In [46]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [47]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [48]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Chandigarh Transport Undertaking (CTU) - 165750,HVAC Seater (2+3),12:20,15:05,02h 45m,INR 195.6,46 Seats available,3.9\n39,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
1,Chandigarh Transport Undertaking (CTU) - 165752,HVAC Seater (2+3),14:20,17:05,02h 45m,INR 195.6,46 Seats available,4.0\n55,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
2,SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA CO.),VE A/C Seater / Sleeper (2+1),20:40,23:30,02h 50m,INR 599,24 Seats available,3.8,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
3,City Land Travels,Volvo A/C Semi Sleeper (2+2),22:15,01:15,03h 00m,889,30 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
4,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),21:00,23:30,02h 30m,INR 599,38 Seats available,3.5,https://www.redbus.in/bus-tickets/yamuna-nagar...,Yamuna Nagar to Chandigarh
...,...,...,...,...,...,...,...,...,...,...
93,Orbit Aviation Pvt. Ltd.,Mercedes Multi-Axle Semi Sleeper (2+2),13:47,18:15,04h 28m,449,29 Seats available,3.8,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Pathankot
94,Khurana Bus Service,Volvo A/C (2+2),14:20,19:30,05h 10m,595,24 Seats available,3.0\n14,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Pathankot
95,Khurana Bus Service,Volvo A/C (2+2),14:25,19:30,05h 05m,510,28 Seats available,4.3\n8,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Pathankot
96,Orbit Aviation Pvt. Ltd.,Mercedes Multi-Axle Semi Sleeper (2+2),14:40,19:00,04h 20m,688,30 Seats available,3.2\n117,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Pathankot


In [49]:
df1.to_csv("C:/Users/Shalini/Downloads/ctu_data6.csv") #done6

In [50]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd
from selenium import webdriver

# Initialize the WebDriver
driver = webdriver.Chrome()

# Load the Redbus homepage
driver.get("https://www.redbus.in")
time.sleep(3)
driver.maximize_window()

# Load the SBSTC page on Redbus
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")
time.sleep(3)

# Function to scrape SBSTC route links and names
def sbstc_link_route(route_xpath):
    LINKS_SBSTC = []  # List to store SBSTC bus links
    ROUTE_SBSTC = []  # List to store SBSTC route names

    # Wait setup
    wait = WebDriverWait(driver, 20)

    try:
        i = 0
        while True:
            # Wait for the route elements on the current page
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_SBSTC.append(route.text)
                LINKS_SBSTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[2]/div[1]/a')))

            # Construct the next button XPath dynamically
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'

            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)

                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")

    return LINKS_SBSTC, ROUTE_SBSTC

# Example route XPath for SBSTC buses (you may need to inspect the page for the exact route element)
route_xpath_sbstc = "//a[@class='route']"

# Calling the function to get links and routes
LINKS_SBSTC, ROUTE_SBSTC = sbstc_link_route(route_xpath_sbstc)

No more pages to paginate at step 0


In [51]:
df_sbstc =pd.DataFrame({"Route_name":ROUTE_SBSTC, "Route_link":LINKS_SBSTC})
df_sbstc

,Route_name,Route_link
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Haldia to Calcutta,https://www.redbus.in/bus-tickets/haldia-to-ko...
3,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
6,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
7,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
8,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [52]:
df=df_sbstc

In [53]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys  # Import Keys
import time
import pandas as pd
from selenium import webdriver

# Initialize the WebDriver
driver_k = webdriver.Chrome()

# Initialize lists for storing data
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

# Loop through each link in your dataframe
for i, r in df.iterrows():
    link = r["Route_link"]
    routes = r["Route_name"]

    # Open the route link
    driver_k.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # Click to view bus details
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
    time.sleep(2)

    # Scroll the page using ActionChains and Keys.PAGE_DOWN
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source

        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()

        time.sleep(5)

        new_page_source = driver_k.page_source

        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)

print("Successfully Completed")

Successfully Completed


In [54]:

# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [55]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,SBSTC-JHALDA - KARUNAMOYEE - VIA - PRL/ASN - 0...,Non AC Seater (2+3),10:45,14:45,04h 00m,INR 155,40 Seats available,4.2\n30,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
1,SBSTC-BAGMUNDI - KOLKATA - 05:00 (BEL DEPOT) -...,Non AC Seater (2+3),10:50,14:20,03h 30m,INR 150,48 Seats available,2.5\n6,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
2,SBSTC-PURULIA - KOLKATA - 07:15 - (VIA ASN) (H...,Non AC Seater (2+3),11:05,14:35,03h 30m,INR 150,52 Seats available,3.8\n7,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
3,SBSTC-DURGAPUR - KARUNAMOYEE - 11:45 (DGP DEPO...,Non AC Seater (2+3),11:45,15:45,04h 00m,INR 155,45 Seats available,3.9\n80,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
4,SBSTC-DURGAPUR - KOLKATA - 12:00 (DGP DEPOT) -...,Non AC Seater (2+3),12:00,15:30,03h 30m,INR 150,52 Seats available,3.9\n42,https://www.redbus.in/bus-tickets/durgapur-to-...,Durgapur to Calcutta
...,...,...,...,...,...,...,...,...,...,...
115,SBSTC-KOLKATA - BANKURA - 14:30 (Bankura Depot...,Non AC Seater (2+3),14:30,18:30,04h 00m,INR 180,51 Seats available,2.8\n23,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura
116,SBSTC-KOLKATA - BANKURA - 15:30 (Bankura Depot...,Non AC Seater (2+3),15:30,19:30,04h 00m,INR 180,52 Seats available,2.2\n27,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura
117,SBSTC-KARUNAMOYEE - BANKURA - VIA - DURGAPUR -...,Non AC Seater (2+3),16:45,22:15,05h 30m,INR 185,29 Seats available,4.4\n68,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura
118,City Queen Bus Service,Non AC Seater (2+2),20:50,02:00,05h 10m,INR 250,13 Seats available,2.3\n5,https://www.redbus.in/bus-tickets/kolkata-to-b...,Kolkata to Bankura


In [56]:
df1.to_csv("C:/Users/Shalini/Downloads/sbstc_data6.csv") #done7

In [57]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from selenium import webdriver

driver = webdriver.Chrome()

# Load the UPSRTC webpage
driver.get("https://www.redbus.in/online-booking/upsrtc")  # UPSRTC Redbus page URL

time.sleep(3)
driver.maximize_window()

def upsrtc_link_route(route_xpath):
    LINKS_UPSRTC = []  # List to store UPSRTC bus links
    ROUTE_UPSRTC = []  # List to store UPSRTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_UPSRTC.append(route.text)
                LINKS_UPSRTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_UPSRTC, ROUTE_UPSRTC

# Example route XPath for UPSRTC buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"  # Adjust according to UPSRTC route elements on the Redbus page

# Calling the function to get links and routes
LINKS_UPSRTC, ROUTE_UPSRTC = upsrtc_link_route(route_xpath)


No more pages to paginate at step 0


In [58]:

df_upsrtc = pd.DataFrame({"Route_name": ROUTE_UPSRTC,"Route_link": LINKS_UPSRTC})

df_upsrtc

,Route_name,Route_link
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
2,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
3,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
4,Lucknow to Allahabad,https://www.redbus.in/bus-tickets/lucknow-to-a...
5,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
6,Delhi to Farrukhabad (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-far...
7,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
8,Badaun to Delhi,https://www.redbus.in/bus-tickets/badaun-to-delhi
9,Allahabad to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...


In [59]:
df=df_upsrtc

In [60]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [61]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [62]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,R S YADAV SMART BUS PRIVATE LIMITED,VE A/C Sleeper (2+1),22:15,04:15,06h 00m,759,29 Seats available,4.4\n91,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
1,New Khaira Transport,A/C Seater / Sleeper (2+1),20:40,03:30,06h 50m,540,9 Seats available,3.1\n11,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
2,Rahul Travels,AshokLeyland Stile A/C,21:00,02:00,05h 00m,629,29 Seats available,1.9\n6,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
3,FlixBus,AC Seater (2+2),23:35,05:00,05h 25m,INR 292.16,16 Seats available,4.2\n30,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
4,SHRI RAM TRAVELS,A/C Seater (2+2),23:59,05:15,05h 16m,INR 740,16 Seats available,3.9\n22,https://www.redbus.in/bus-tickets/delhi-to-bar...,Delhi to Bareilly
...,...,...,...,...,...,...,...,...,...,...
96,Chaturvedi Bus Service,Non A/C Seater / Sleeper (2+1),20:00,05:20,09h 20m,INR 400,45 Seats available,New,https://www.redbus.in/bus-tickets/farrukhabad-...,Farrukhabad (Uttar Pradesh) to Delhi
97,Chaturvedi Bus Service,A/C Seater/Sleeper (2+1),19:40,04:45,09h 05m,INR 2000,47 Seats available,New,https://www.redbus.in/bus-tickets/farrukhabad-...,Farrukhabad (Uttar Pradesh) to Delhi
98,Yuvraj Travels,NON AC Seater / Sleeper 2+1,18:30,06:00,11h 30m,INR 550,40 Seats available,4.1,https://www.redbus.in/bus-tickets/farrukhabad-...,Farrukhabad (Uttar Pradesh) to Delhi
99,Shree Krishna Shtabadi Travels,NON AC Seater / Sleeper 2+1,18:30,05:20,10h 50m,INR 600,44 Seats available,1.4,https://www.redbus.in/bus-tickets/farrukhabad-...,Farrukhabad (Uttar Pradesh) to Delhi


In [63]:
df1.to_csv("C:/Users/Shalini/Downloads/upsrtc_data.csv") #done8

In [64]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from selenium import webdriver

driver = webdriver.Chrome()

# Load the PEPSU webpage
driver.get("https://www.redbus.in/online-booking/pepsu")  # PEPSU Redbus page URL

time.sleep(3)
driver.maximize_window()

def pepsu_link_route(route_xpath):
    LINKS_PEPSU = []  # List to store PEPSU bus links
    ROUTE_PEPSU = []  # List to store PEPSU route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_PEPSU.append(route.text)
                LINKS_PEPSU.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_PEPSU, ROUTE_PEPSU

# Example route XPath for PEPSU buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"  # Adjust according to PEPSU route elements on the Redbus page

# Calling the function to get links and routes
LINKS_PEPSU, ROUTE_PEPSU = pepsu_link_route(route_xpath)

No more pages to paginate at step 0


In [65]:
df_pepsu = pd.DataFrame({"Route_name": ROUTE_PEPSU,"Route_link": LINKS_PEPSU})
df_pepsu

,Route_name,Route_link
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
2,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
3,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
5,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
6,Delhi Airport to Patiala,https://www.redbus.in/bus-tickets/delhi-airpor...
7,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...
8,Delhi Airport to Ludhiana,https://www.redbus.in/bus-tickets/delhi-airpor...
9,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...


In [66]:
df=df_pepsu

In [67]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [68]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [69]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Yolo Bus,VE A/C Seater / Sleeper (2+1),23:30,05:24,05h 54m,INR 690,11 Seats available,4.6\n380,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
1,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),22:20,05:00,06h 40m,1587,6 Seats available,4.5\n329,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
2,Yolo Bus,VE A/C Seater / Sleeper (2+1),22:31,05:01,06h 30m,INR 650,17 Seats available,4.2\n96,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
3,Laxmi holidays,VE A/C Sleeper (2+1),23:00,06:00,07h 00m,1235,17 Seats available,4.4\n342,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
4,Zingbus Plus,A/C Seater / Sleeper (2+1),22:05,05:40,07h 35m,893,21 Seats available,4.8\n33,https://www.redbus.in/bus-tickets/delhi-to-lud...,Delhi to Ludhiana
...,...,...,...,...,...,...,...,...,...,...
71,PEPSU (Punjab) - 67096,Volvo AC Seater (2+2),15:03,00:05,09h 02m,INR 1169,35 Seats available,3.2\n9,https://www.redbus.in/bus-tickets/jalandhar-to...,Jalandhar to Delhi Airport
72,PEPSU (Punjab) - 67135,Volvo AC Seater (2+2),15:33,02:35,11h 02m,INR 1169,32 Seats available,4.2\n5,https://www.redbus.in/bus-tickets/jalandhar-to...,Jalandhar to Delhi Airport
73,PEPSU (Punjab) - 67098,Volvo AC Seater (2+2),18:10,04:05,09h 55m,INR 1169,32 Seats available,4.0\n11,https://www.redbus.in/bus-tickets/jalandhar-to...,Jalandhar to Delhi Airport
74,PEPSU (Punjab) - 67128,Volvo AC Seater (2+2),19:10,04:50,09h 40m,INR 1169,33 Seats available,4.7\n14,https://www.redbus.in/bus-tickets/jalandhar-to...,Jalandhar to Delhi Airport


In [70]:
df1.to_csv("C:/Users/Shalini/Downloads/pepsu_data.csv") #done9

In [71]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from selenium import webdriver

driver = webdriver.Chrome()

# Load the WBTC webpage
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")  # WBTC Redbus page URL

time.sleep(3)
driver.maximize_window()

def wbtc_link_route(route_xpath):
    LINKS_WBTC = []  # List to store WBTC bus links
    ROUTE_WBTC = []  # List to store WBTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_WBTC.append(route.text)
                LINKS_WBTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_WBTC, ROUTE_WBTC

# Example route XPath for WBTC buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"

# Calling the function to get links and routes
LINKS_WBTC, ROUTE_WBTC = wbtc_link_route(route_xpath)


No more pages to paginate at step 0


In [72]:
df_wbtc = pd.DataFrame({"Route_name": ROUTE_WBTC,"Route_link": LINKS_WBTC})
df_wbtc

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
4,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
5,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri
6,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
7,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
8,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...


In [73]:
df=df_wbtc

In [74]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [75]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [76]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,WBTC (CTC) HABRA-DIGHA via Bally - 26|13:25,Non AC Seater (2+3),13:25,18:10,04h 45m,INR 161,51 Seats available,3.0\n34,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
1,WBTC (CTC) Barasat-Digha Via Bally (AC VOLVO) ...,Volvo AC Seater (2+2),14:00,18:00,04h 00m,INR 410,34 Seats available,4.5\n38,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
2,"WBTC (CTC) BARASAT-DIGHA via Esplanade, Karuna...",Non AC Seater (2+3),14:15,19:15,05h 00m,INR 165,43 Seats available,3.4,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
3,WBTC (CTC) HABRA-DIGHA via Bally - 26|14:50,Non AC Seater (2+3),14:50,19:35,04h 45m,INR 161,50 Seats available,3.5\n49,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
4,WBTC (CTC) MADHYAMGRAM - DIGHA via Belghoria -...,Non AC Seater (2+3),16:00,21:05,05h 05m,INR 161,33 Seats available,2.3\n24,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
5,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,03:35,06h 20m,520,52 Seats available,3.7,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
6,Aradhana Bus Service,Non AC Seater (2+3),16:50,23:30,06h 40m,INR 350,54 Seats available,2.1,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
7,Aradhana Travels,A/C Seater / Sleeper (2+2),18:35,23:55,05h 20m,463,19 Seats available,1.1\n10,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
8,Satya Paribahan,A/C Seater (2+3),23:00,05:10,06h 10m,INR 320,41 Seats available,2.8\n32,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)
9,Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,04:30,05h 30m,INR 550,71 Seats available,2.1\n6,https://www.redbus.in/bus-tickets/digha-to-bar...,Digha to Barasat (West Bengal)


In [77]:
df1.to_csv("C:/Users/Shalini/Downloads/wbtc_data.csv") #done10

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from selenium import webdriver

driver = webdriver.Chrome()

# Load the BSRTC webpage
driver.get("https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile")  # BSRTC Redbus page URL

time.sleep(3)
driver.maximize_window()

def bsrtc_link_route(route_xpath):
    LINKS_BSRTC = []  # List to store BSRTC bus links
    ROUTE_BSRTC = []  # List to store BSRTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_BSRTC.append(route.text)
                LINKS_BSRTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_BSRTC, ROUTE_BSRTC

# Example route XPath for BSRTC buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"  # Adjust according to BSRTC route elements on the Redbus page

# Calling the function to get links and routes
LINKS_BSRTC, ROUTE_BSRTC = bsrtc_link_route(route_xpath)


No more pages to paginate at step 0


In [6]:

df_bsrtc = pd.DataFrame({"Route_name": ROUTE_BSRTC,"Route_link": LINKS_BSRTC})

df_bsrtc

,Route_name,Route_link
0,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
2,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...
3,Motihari to Delhi,https://www.redbus.in/bus-tickets/motihari-to-...
4,Bettiah to Patna (Bihar),https://www.redbus.in/bus-tickets/bettiah-to-p...
5,Delhi to Motihari,https://www.redbus.in/bus-tickets/delhi-to-mot...
6,Balmiki Nagar (bihar) to Patna (Bihar),https://www.redbus.in/bus-tickets/balmiki-naga...
7,Patna (Bihar) to Balmiki Nagar (bihar),https://www.redbus.in/bus-tickets/patna-to-bal...
8,Patna (Bihar) to Purnea,https://www.redbus.in/bus-tickets/patna-to-purnea
9,Patna (Bihar) to Katihar,https://www.redbus.in/bus-tickets/patna-to-kat...


In [7]:
df=df_bsrtc

In [8]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [9]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [10]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,16:30,08:30,16h 00m,1020,31 Seats available,3.1\n362,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
1,Bihar state road transport corporation (BSRTC)...,Volvo AC Seater 2+2,19:10,11:59,16h 49m,1020,36 Seats available,3.9\n123,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
2,ROYAL TRAVELS AND CARGO,A/C Seater / Sleeper (2+2),17:30,08:30,15h 00m,INR 696,32 Seats available,2.2\n56,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
3,Panwar tour and Travels,A/C Seater / Sleeper (2+2),15:40,09:30,17h 50m,INR 1550,19 Seats available,3.2\n7,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
4,Rajdhani Express,A/C Seater / Sleeper (2+2),15:00,07:00,16h 00m,924,30 Seats available,1.0\n5,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
5,Travel Point World LLP,A/C Seater / Sleeper (2+2),20:20,10:50,14h 30m,1749,38 Seats available,1.9\n6,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
6,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00,11:49,16h 49m,INR 1050,56 Seats available,1.3\n11,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
7,Singh Express Bus Service,A/C Seater / Sleeper (2+2),15:00,10:00,19h 00m,INR 1599,20 Seats available,1.0\n6,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
8,Shree Balajee Travels And Cargo,A/C Seater / Sleeper (2+2),20:00,14:30,18h 30m,1079,72 Seats available,1.2,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi
9,Panwar tour and Travels,A/C Seater / Sleeper (2+2),15:40,09:30,17h 50m,INR 1750,19 Seats available,3.2,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Delhi


In [11]:
df1.to_csv("C:/Users/Shalini/Downloads/bsrtc_data01.csv") #done11

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
from selenium import webdriver

driver = webdriver.Chrome()

# Load the ASTC webpage
driver.get("https://www.redbus.in/online-booking/astc")  # ASTC Redbus page URL

time.sleep(3)
driver.maximize_window()

def astc_link_route(route_xpath):
    LINKS_ASTC = []  # List to store ASTC bus links
    ROUTE_ASTC = []  # List to store ASTC route names

    # Wait setup
    wait = WebDriverWait(driver, 10)

    try:
        i = 0
        while True:
            # Wait for the route elements to be present
            paths = wait.until(EC.presence_of_all_elements_located((By.XPATH, route_xpath)))

            # Retrieve names of the routes and their corresponding links
            for route in paths:
                ROUTE_ASTC.append(route.text)
                LINKS_ASTC.append(route.get_attribute("href"))  # Assuming it's an <a> tag

            # Wait for the pagination element to be present (update the XPath if needed)
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Construct the next button XPath dynamically (adjust according to actual button structure)
            next_button_xpath = f'//*[@id="root"]/div/div[4]/div[12]/div[1]//a[text()="{i+1}"]'
            
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling
                
                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript
                
                i += 1  # Move to the next pagination step
            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Encountered an issue at step {i}: {e}")
    
    return LINKS_ASTC, ROUTE_ASTC

# Example route XPath for ASTC buses (you may need to inspect the page for the exact route element)
route_xpath = "//a[@class='route']"  # Adjust according to ASTC route elements on the Redbus page

# Calling the function to get links and routes
LINKS_ASTC, ROUTE_ASTC = astc_link_route(route_xpath)


No more pages to paginate at step 0


In [13]:

# Create a DataFrame for ASTC
df_astc = pd.DataFrame({"Route_name": ROUTE_ASTC,"Route_link": LINKS_ASTC})
# Display the DataFrame
df_astc


,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
3,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
7,Jorhat to Dibrugarh,https://www.redbus.in/bus-tickets/jorhat-to-di...
8,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...
9,North Lakhimpur to Sibsagar,https://www.redbus.in/bus-tickets/north-lakhim...


In [14]:
df = df_astc

In [15]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [16]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df1 = pd.DataFrame(data)
#convert dataframe to csv


In [17]:
df1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,Assam State Transport Corporation (ASTC) - 170878,Bharat Benz A/C Seater (2+2),10:35,15:15,04h 40m,INR 298,20 Seats available,4.6\n81,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
1,Assam State Transport Corporation (ASTC) - 157959,Bharat Benz A/C Seater (2+2),14:00,18:30,04h 30m,INR 298,24 Seats available,4.2\n206,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
2,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,20:00,04h 15m,INR 298,21 Seats available,4.0\n53,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
3,Assam State Transport Corporation (ASTC) - 161055,Volvo AC Seater 2+2,16:15,20:30,04h 15m,INR 298,19 Seats available,4.3\n80,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
4,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),10:10,15:00,04h 50m,INR 450,34 Seats available,3.1\n18,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
5,Chartered Bus - ASTC,A/C Seater Push Back (2+2),11:30,16:15,04h 45m,INR 334.68,15 Seats available,4.1\n110,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
6,Chartered Bus - ASTC,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),13:00,17:00,04h 00m,INR 389.13,30 Seats available,4.7\n483,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
7,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,14:30,18:30,04h 00m,INR 432.06,32 Seats available,4.6\n118,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
8,SHIVAM TRAVELS,Bharat Benz A/C Seater (2+1),15:00,19:15,04h 15m,324,32 Seats available,4.0\n196,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati
9,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,15:30,19:15,03h 45m,INR 389.13,33 Seats available,4.6\n488,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Tezpur to Guwahati


In [18]:
df1.to_csv("C:/Users/Shalini/Downloads/astc_data.csv") #done12